# <center>Prediction of current clamp from voltage clamp parameters <br> in different layers and cre types</center>
<center>Corinne Teeter</center>
<center>Jan 4, 2019</center>

### Objective 
Determine if parameters (amp, rise_tau, latency, and decay_tau) and fitting error metric (NRMSE) found by fitting current clamp data can be predicted from parameters found by fitting voltage clamp data (using fits to the average first pulse trace).  If so, do these relationships differ between excitation, different cre lines, or layers?
### Results
1) Decay tau is not well fit and is left out of the analysis as regressions are not found to be statistically significant. <br>

2) 'Excellent' versus 'well' fit data: It is not obvious whether including the additional 'well' fit data is benificial i.e. sometimes it detracts from statistical significance and sometime it enhances it. <br>

3) Excitatory versus Inhibitory: It is reasonable to assume that there are relationships between all parameters in voltage clamp and current clamp and that they are different (statistically significant) between inhibitory and excitatory neurons.  Rise tau is the most substantially different and noticible by eye. <br>

4) Cre-lines: Included catagories are pvalb to pvalb, sim1 to sim1, tlx3 to tlx3, rorb to rorb, and unknown to unknown.  These catagories potentially had a large enough number to have statistically significant results; however, tlx3 to tlx3 and rorb to rorb sill had low n, which could have contributed to a lack of significance.  There are many details in these results.  Different cre line catagories have different levels of significance for different variables.  Using only significant regressions, only intercepts (never slopes) were found to be statistically different in some variables.  Rise tau that was found to be highly significantly different between inhibitory and excitatory cells was not statistically significant here.  However by eye it is quite obvious that the unknown to unknown and the sim1 to sim1 (which were the two catagories that have significant regression lines for rise_tau) are located above the other data which were not individually statistically significant. <br>

5) Layers: Included layer catagories are 2 to 2, 2/3 to 2/3, 3 to 3, 4 to 4, 5 to 5, and 6 to 6.  Only the intercept of the rise tau relationship was found to be statistically significant.
### Methods
a) Fit the average of the first pulse voltage clamp and current clamp data. <br>
b) Rate the quality of the fit by eye. <br>
c) Remove heteroscedasticity. <br>
d) Break into catagories (i.e. inhibitory/excitatory, cre line, layer). <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;test for statistical significance of linear regression. <br>
e) Test if statistically significant regressions (slope and intercept) are different between different catagories. <br>  
### Subquestions addressed in this analysis
What is the best way to get rid of heteroscedasticity? <br>
### Caveats
a) In many of the fits, it does not look like rise is fit well with one exponential.  This throws off the location of the amplitude. <br>
b) Low n in some samples.
### Discussion topics
Why are NRMSE, amplitude and rise_tau heteroscedastic? <br>
What do the different predictive relationships between different catagories mean? <br>
What is the interpretation of a statistically significant different slope versus intercept.
### Potenial future work
Use double exponentials in rise tau (and maybe decay tau) to get better fits. <br>
Stats: <br>
&nbsp;&nbsp;&nbsp;Test for outliers <br>
Make models of voltage/current clamp relationships.


# ANALYSIS

In [ ]:
# Note that these .csv files loaded here have been processed though "extract_first_pulse_fit_data_from_DB.py"
# "and catagorize_goodness_of_fit_by_eye.py"
# A fantastic explanation of how to interpret the output and input of linear regression
# of different catagories in statsmodels (or R) is at: 
# https://www.andrew.cmu.edu/user/achoulde/94842/lectures/lecture10/lecture10-94842.html
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
font={'size':22}
matplotlib.rc('font', **font)
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
# load csv files.
i_df=pd.read_csv('ML_connected_iclamp_2018_12_18.csv')
v_df=pd.read_csv('ML_connected_vclamp_2018_12_12.csv')
i_df['uid']=i_df.apply(lambda row: "%.3f" % float(row.uid), axis=1)
v_df['uid']=v_df.apply(lambda row: "%.3f" % float(row.uid), axis=1)

In [ ]:
# get rid of wacky Unnamed columns if they exist
v_df=v_df[v_df.columns.drop(list(v_df.filter(regex='Unnamed')))]
i_df=i_df[i_df.columns.drop(list(i_df.filter(regex='Unnamed')))]
i_df.keys()

In [ ]:
#Merge voltage and current clamp data frames
merged_df = pd.merge(i_df, v_df, on=['uid', 'pre_cell_id', 'post_cell_id', 
                                     'distance', 'acsf','post_cre', 'pre_cre',
                                    'boolean_connection', 'pre_layer', 
                                     'post_layer'], how='inner', suffixes={'_i', '_v'})
merged_df['uid']=merged_df['uid'].astype(str)

print(len(i_df))
print(len(v_df))
print(len(merged_df))

# Data Sets: excellent versus well fit data
The quality of the fit was assessed by eye. See images at the end of the document for examples.  More data will be included in this analysis if the 'well fit' data are used in addition to the 'excellent fit' data.  However, it is unclear if including well fit data will benifit the analysis as it may add  noise.  Throughout, results on the 'excellent' versus 'well' fit data will be shown.  The decay tau values are not yet well fit and many are hitting the boundries.  Here, the values hitting the boundry in vclamp are removed, however, this does not resolve the issue, as in general, decay tau is not well fit. 

In [ ]:
# merge the data that is 'excellent'
excellent_df=merged_df[(merged_df['good_fit_i']=='excellent') & (merged_df['good_fit_v']=='excellent') & 
                      (merged_df['data_clarity_v']=='well') & (merged_df['data_clarity_i']=='well')]
# merge the data that is pretty good
good_df=merged_df[((merged_df['good_fit_i']=='excellent') | (merged_df['good_fit_i']=='good')) & 
                       ((merged_df['good_fit_v']=='excellent') | (merged_df['good_fit_v']=='good')) & 
                       ((merged_df['data_clarity_v']=='well') | (merged_df['data_clarity_v']=='ok')) & 
                       ((merged_df['data_clarity_i']=='well') | (merged_df['data_clarity_i']=='ok'))]
print(len(excellent_df))

# Heteroscedasticity
Heteroscedasticity is when the variability of a variable is unequal across the range of values (often it systematically increases or decreases).  It can be seen by eye in this data.  Here, I explore if two data transforms will abolish heteroscedasticity.  The White’s Lagrange Multiplier test and Breusch–Pagan test for Heteroscedasticity provide two measures with p-values to assess whether the heteroscedasticity is statistically significant.  The  Breusch–Pagan test only tests for linear forms of heteroscedasticity.  The White test idendifies non-linear forms of heteroscedasticity.  Note that in transforms of the amplitude data, negative numbers are converted to positive numbers.  Green plots denote when both statistical tests show no significant heteroscedasticity,  yellow plots denote when one tests yields statistical significance for heteroscedasticity, and red is when both tests claim heteroscedasticity.  
### Code

In [ ]:
def pos_sqrt(n):
    """Take square root of the absolute value of a number.  If the input is zero, return a np.nan.
    
    Input
    -----
    n: float
    
    Returns
    -------
    out: float"""
    
    if n<0:
        out = np.sqrt(np.abs(n))
    if n>=0:
        out = np.sqrt(n)
    if np.isnan(n):
        return np.nan
    return out

def pos_log(n):
    """Take log of the absolute value of a number.  If the input is zero, return a np.nan.
    
    Input
    -----
    n: float
    
    Returns
    -------
    out: float"""
    if n<0:
        out = np.log(np.abs(n))
    if n>=0:
        out = np.log(n)
    if np.isnan(n):
        return np.nan
    return out

def plot_hetero(df):
    """Plot the original data with two data transforms (sqrt and log) and perform tests 
    for heterscedasticity.
    
    Input
    -----
    df: DataFrame
    
    Output
    ------
    df: DataFrame
        contains additional columns with the values for the transforms
    """
    variables=(('NRMSE_v', 'NRMSE_i'),
               ('amp_v', 'amp_i'),
               ('rise_time_v', 'rise_time_i'),
               ('latency_v', 'latency_i'),
               ('decay_tau_v', 'decay_tau_i'))

    def set_plot_color(gca, p1, p2, sig_v=.05):
        if p1<sig_v and p2<sig_v:
            gca.set_facecolor('xkcd:red') #significant p-value for heteroscedasticity
        elif p1<sig_v or p2<sig_v:
            gca.set_facecolor('xkcd:yellow') #one p-value significant for heteroscedasticity
        elif p1>sig_v and p2>sig_v:
            gca.set_facecolor('xkcd:green') #neither p-values suggest heteroscedasticity
            

    def plotting(var, df): 
        # plot data on left
        f.add_subplot(131)
        regression=smf.ols(formula='%s ~ %s' %(var[1], var[0]), data=df).fit() 
        _,_,_,p_unchange_white=statsmodels.stats.diagnostic.het_white(regression.resid, regression.model.exog)
        _,_,_,p_unchange_breuschpagan=statsmodels.stats.diagnostic.het_breuschpagan(regression.resid, regression.model.exog)
        sns.regplot(var[0], var[1], data=df, color ='k', label='p_white=%f, p_breuschpagan=%f' % (p_unchange_white, p_unchange_breuschpagan))
        set_plot_color(plt.gca(), p_unchange_white, p_unchange_breuschpagan)
        plt.xlim([np.min(df[var[0]]), np.max(df[var[0]])])
        plt.ylim([np.min(df[var[1]]), np.max(df[var[1]])])
        plt.legend()

        # plot square root transformed data on right
        f.add_subplot(132)
        df['sqrt_'+var[0]]=df[var[0]].apply(lambda x: pos_sqrt(x))
        df['sqrt_'+var[1]]=df[var[1]].apply(lambda x: pos_sqrt(x))
        regression=smf.ols(formula='%s ~ %s' %('sqrt_'+var[1], 'sqrt_'+var[0]), data=df).fit() 
        _,_,_,p_sqrt_white=statsmodels.stats.diagnostic.het_white(regression.resid, regression.model.exog)
        _,_,_,p_sqrt_breuschpagan=statsmodels.stats.diagnostic.het_breuschpagan(regression.resid, regression.model.exog)
        sns.regplot('sqrt_'+var[0], 'sqrt_'+var[1], data=df, color ='k', label='p_white=%f, p_breuschpagan=%f' % (p_sqrt_white, p_sqrt_breuschpagan))
        set_plot_color(plt.gca(), p_sqrt_white, p_sqrt_breuschpagan)
        plt.xlim([np.min(df['sqrt_'+var[0]]), np.max(df['sqrt_'+var[0]])])
        plt.ylim([np.min(df['sqrt_'+var[1]]), np.max(df['sqrt_'+var[1]])])
        plt.legend()

        # plot log transformed data on the far right
        f.add_subplot(133)
        df['log_'+var[0]]=df[var[0]].apply(lambda x: pos_log(x))
        df['log_'+var[1]]=df[var[1]].apply(lambda x: pos_log(x))
        regression=smf.ols(formula='%s ~ %s' %('log_'+var[1], 'log_'+var[0]), data=df).fit() 
        _,_,_,p_log_white=statsmodels.stats.diagnostic.het_white(regression.resid, regression.model.exog)
        _,_,_,p_log_breuschpagan=statsmodels.stats.diagnostic.het_breuschpagan(regression.resid, regression.model.exog)
        sns.regplot('log_'+var[0], 'log_'+var[1], data=df, color ='k', label='p_white=%f, p_breuschpagan=%f' % (p_log_white, p_log_breuschpagan))
        set_plot_color(plt.gca(), p_log_white, p_log_breuschpagan)
        plt.xlim([np.min(df['log_'+var[0]]), np.max(df['log_'+var[0]])])
        plt.ylim([np.min(df['log_'+var[1]]), np.max(df['log_'+var[1]])])
        plt.legend()

    for var in variables:
        font={'size':22}
        matplotlib.rc('font', **font)
        f=plt.figure(figsize=(40,10))
        if var==('decay_tau_v', 'decay_tau_i'):
            plotting(var,df[df['decay_tau_v']<.49])
        else:
            plotting(var, df)

        plt.show()
        
    return df

## Heteroscedasticity in 'excellent fit' data set
### Results
Amplitude, rise tau, and NMRSE require log transform to remove heteroscedasticity (p-value is less significant).  Interestingly the linear heteroscedasticity test (Breusch–Pagan) claims heteroscedasticity is present by the non-linear test does not.  I will go ahead and use the log transform when performing statistical tests later in this document. Latency and decay tau do not require a transform.  The following analysis testing catagorical regression significance and differences will incorporate the necessary transforms.

In [ ]:
excellent_df=plot_hetero(excellent_df)

## Plot the 'well fit' (larger) data set
### Results
Similar to 'excellent fit' data set, NRMSE, rise_time, and amplitude require log transforms to remove heteroscedasticity and decay tau does not require a transform.  The one difference here, is that the latency value shows a statistically significant p-value for heteroscedasticity when there is no transform, non significant with a sqrt transform and sigfificant again with a log transform.  I am going to ignore this deviation from the 'excellent fit' data since the data does not look blatently heteroscedastic, the p-value is only passes 95% significance, and it may be the case that one value has triggered the significant p-value here (although it is really unclear which data point that would have been as it is not obvious by eye). 

In [ ]:
good_df=plot_hetero(good_df)

# Functions for regression significance and catagorical differences used in analysis below
### Code to test if the data can be fit via a regression

In [ ]:
def regression_significance(df_list, catagory_type, data_type='', 
                            variables=(('NRMSE_v', 'NRMSE_i'),
                                       ('amp_v', 'amp_i'),
                                       ('rise_time_v', 'rise_time_i'),
                                       ('latency_v', 'latency_i'),
                                       ('decay_tau_v', 'decay_tau_i'))):
    """Creates plots that show whether variables can be fit with linear regression lines for
    different catagories. 
    
    Input
    -----
    df_list: list of DataFrames
        results from each DataFrame will be plotted side by side
    catagory_type: string
        options: excitation, cre, layer
    data_type: string
    data_type: string
        specifies which data transform you are using.
        options: ''       non transformed data
                 'log_'   sqrt of data (negative values made positive before transform)
                 'sqrt_'  log of data (negative values made positive before transform)
    variables: tuple of tuples containing string pairs
        string pairs are column titles to be plotted against one another.  
    """
    df_num=len(df_list)
    font={'size':22/df_num}
    matplotlib.rc('font', **font)

    
    if catagory_type=='excitation':
        col_name=('syn_excitation_v', 'syn_excitation_v')
        values=(('ex', 'ex', 'b'),
                ('in', 'in', 'r'))
        
    elif catagory_type=='cre':
        col_name=('pre_cre', 'post_cre')
        values=(('pvalb', 'pvalb', 'b'),
                ('rorb', 'rorb', 'r'),
                ('sim1', 'sim1', 'g'),
                ('tlx3','tlx3', 'm'),
                ('unknown', 'unknown', 'c'))
    elif catagory_type=='layer':
        col_name=('pre_layer', 'post_layer')
        values=(('2', '2','b'),
               ('2/3', '2/3', 'r'),
               ('3', '3', 'g'),
               ('4', '4', 'm'),
               ('5', '5', 'c'),
               ('6', '6', 'y'))
    else:
        raise Exception()
    
#     variables=(('NRMSE_v', 'NRMSE_i'),
#                ('amp_v', 'amp_i'),
#                ('rise_time_v', 'rise_time_i'),
#                ('latency_v', 'latency_i'),
#                ('decay_tau_v', 'decay_tau_i'))

    fs=20.                  
    def plotting(var, df_list):
        for ii,df in enumerate(df_list):
            plt.subplot(1,df_num, ii+1)
            mod = smf.ols(formula='%s ~ %s' % (data_type+var[1], data_type+var[0]), data=df)
            res=mod.fit()
            if res.pvalues[1]< .05:
                slope_star='*'
            else:
                slope_star=''
            sns.regplot(data_type+var[0], data_type+var[1], data=df, fit_reg=True, color ='k', 
                            label='all, n=%i, p_slope=%f%s' % 
                            (len(df), res.pvalues[1], slope_star))
            for value in values: 
                plot_df=df[(df[col_name[0]]==value[0]) & (df[col_name[1]]==value[1])]
                mod = smf.ols(formula='%s ~ %s' % (data_type+var[1], data_type+var[0]), data=plot_df)
                res=mod.fit()
                if res.pvalues[1]< .05:
                    slope_star='*'
                else:
                    slope_star=''
                sns.regplot(data_type+var[0], data_type+var[1], data=plot_df, fit_reg=True, color=value[2], 
                            label='%s to %s, n=%i, slope=%f, p_slope=%f%s' % 
                            (value[0], value[1], len(plot_df),res.params[1], res.pvalues[1], slope_star))
    #             sns.regplot(var[0], var[1], data=plot_df, fit_reg=True, color=value[2], 
    #                         label='%s to %s, n=%i, slope=%.2E, int=%.2E' % 
    #                         (value[0], value[1], len(plot_df), res.params[var[0]], res.params.Intercept))
            plt.xlim([np.min(df[data_type+var[0]]), np.max(df[data_type+var[0]])])
            plt.ylim([np.min(df[data_type+var[1]]), np.max(df[data_type+var[1]])])
            plt.legend()
    
    for var in variables:
        plt.figure(figsize=(fs,fs/df_num))
        if var==('decay_tau_v', 'decay_tau_i'):
            sub_df_list=[]
            for df in df_list:
                new=df[df['decay_tau_v']<.49]
                sub_df_list.append(new)
            plotting(var,sub_df_list)
        else:
            plotting(var, df_list)        
        plt.show()
        
# regression_significance([excellent_df, good_df], 'excitation', data_type='log_', variables=(('NRMSE_v', 'NRMSE_i'),
#                ('amp_v', 'amp_i'),
#                ('rise_time_v', 'rise_time_i')))
# regression_significance([excellent_df, good_df], 'excitation', data_type='', variables=(
#                ('latency_v', 'latency_i'),
#                ('decay_tau_v', 'decay_tau_i')))    

### Code to test if linear regressions of different catagories are statistically different 

In [ ]:
def cat_sig_diff(df_list, catagory_type, data_type='', variables=(('NRMSE_v', 'NRMSE_i'),
                                       ('amp_v', 'amp_i'),
                                       ('rise_time_v', 'rise_time_i'),
                                       ('latency_v', 'latency_i'),
                                       ('decay_tau_v', 'decay_tau_i'))):
    """
    Create plots showing whether catagories have statistically different linear regressions
    Inputs
    ------
    df: pandas DataFrame
        should be proccessed from catagory_df
    catagory_type: string
        specifies which catagory to apply. Options are 'cre','layer', 'excitation'.
    data_type: string
        specifies which data transform you are using.
        options: ''       non transformed data
                 'log_'   sqrt of data (negative values made positive before transform)
                 'sqrt_'  log of data (negative values made positive before transform)
    variables: tuple of tuples containing string pairs
        string pairs are column titles to be plotted against one another.      
    """

    df_num=len(df_list)
    font={'size':22/df_num}
    matplotlib.rc('font', **font)
    if catagory_type=='excitation':
        col_name='syn_excitation_v'
    elif catagory_type=='cre':
        col_name='cre_catagory'
    elif catagory_type=='layer':
        col_name='layer_catagory'
    else:
        raise Exception('catagory doesnt exist')
    
    fs=20.
    def plotting(var, df_list):
        for ii,df in enumerate(df_list):
            plt.subplot(1,df_num, ii+1)
            # each catagory is allowed independent intercepts and slopes
            model=smf.ols(formula='%s ~ %s * %s' %(data_type+var[1], col_name, data_type+var[0]), data=df).fit()
            # all data lumped together
            int_same_model=smf.ols(formula='%s ~ %s' %(data_type+var[1], data_type+var[0]), data=df).fit() 
            # each catagory is allowed independent intercepts
            int_diff_model=smf.ols(formula='%s ~ %s + %s' %(data_type+var[1],col_name, data_type+var[0]), data=df).fit()

            # Note that simplier model must go first in the sm.satats.anova_lm!!!!
            # check to see if the intercepts are significantly different
            p_int=sm.stats.anova_lm(int_same_model, int_diff_model).values[-1][-1]
            # check to see if slopes are significantly different
            p_slope=sm.stats.anova_lm(int_diff_model, model).values[-1][-1]

            # plot catagories for visual even though individual statistics aren't needed for global stats
            for value in df[col_name].unique(): 
                plot_df=df[df[col_name]==value]
    #             mod = smf.ols(formula='%s ~ %s' % (data_type+var[1], data_type+var[0]), data=plot_df)
    #             res=mod.fit()
                sns.regplot(data_type+var[0], data_type+var[1], data=plot_df, fit_reg=True, 
                            label='%s, n=%i' % (value, len(plot_df)))
            plt.xlim([np.min(df[data_type+var[0]]), np.max(df[data_type+var[0]])])
            plt.ylim([np.min(df[data_type+var[1]]), np.max(df[data_type+var[1]])])
            if p_int<.05:
                int_star='*'
            else:
                int_star=''
            if p_slope<.05:
                slope_star='*'
            else:
                slope_star=''
                    
            plt.title("Stat diff: intercept p=%f%s, slope p=%f%s" % (p_int,int_star, p_slope, slope_star))
            plt.legend()
    
    for var in variables:
        plt.figure(figsize=(fs,fs/df_num))
        if var==('decay_tau_v', 'decay_tau_i'):
            sub_df_list=[]
            for df in df_list:
                new=df[df['decay_tau_v']<.49]
                sub_df_list.append(new)
            plotting(var,sub_df_list)
        else:
            plotting(var, df_list)        
        plt.show()
    
# cat_sig_diff([excellent_df, good_df], 'excitation', data_type='log_', variables=(('NRMSE_v', 'NRMSE_i'),
#                ('amp_v', 'amp_i'),
#                ('rise_time_v', 'rise_time_i')))      
# cat_sig_diff([cre_cat_excellent_df, cre_cat_good_df],'excitation', data_type='', variables=(
#                ('latency_v', 'latency_i'),))


### Code to reduce DataFrames to catagories to be tested

In [ ]:
def create_catagory_df(df, catagory_type, variable, fit_type):
    ''' To test if the different catagories are significantly different catagorical variables 
    that are non significant which should not be removed from the DataFrame.
    
    Input
    -----
    df: Pandas DataFrame
    catagory_type: string
        specifies which catagory to apply. Options are 'cre','layer', 'excitation'.
    variable: string
        options: NRMSE
    fit_type: string
        options: 
            'excellent': excellent fit data set
            'good': well fit data set which also includes excellent data
            
    Returns
    -------
    new_df: Pandas DataFrame
        Includes only a subset of the data to be used by cat_sig_diff
    '''
    if (fit_type != 'excellent') and (fit_type!='good'):
        raise Exception('fit_type not defined')
    
    if catagory_type=='cre':
        col_name=('pre_cre', 'post_cre')
        if variable=='NRMSE':
            if fit_type=='excellent':
                values=(('pvalb', 'pvalb'),
                        ('sim1', 'sim1'),
                        ('tlx3','tlx3'),
                        ('unknown', 'unknown'))
            elif fit_type=='good':
                values=(('pvalb', 'pvalb'),
                        ('sim1', 'sim1'),
                        ('unknown', 'unknown'))        
        elif variable=='amp':
            if fit_type=='excellent':
                values=(('pvalb', 'pvalb'),
                        ('sim1', 'sim1'),
                        ('unknown', 'unknown'))
            elif fit_type=='good':
                values=(('pvalb', 'pvalb'),
                        ('rorb', 'rorb'),
                        ('sim1', 'sim1'),
                        ('tlx3','tlx3'),
                        ('unknown', 'unknown'))
        elif variable=='rise_time':
            if fit_type=='excellent':
                values=(('sim1', 'sim1'),
                        ('unknown', 'unknown'))
            elif fit_type=='good':
                values=(('sim1', 'sim1'),
                        ('unknown', 'unknown'))
        elif variable=='latency':
            if fit_type=='excellent':
                values=(('pvalb', 'pvalb'),
                        ('rorb', 'rorb'),
                        ('sim1', 'sim1'),
                        ('tlx3','tlx3'),
                        ('unknown', 'unknown'))
            elif fit_type=='good':
                values=(('pvalb', 'pvalb'),
                        ('rorb', 'rorb'),
                        ('sim1', 'sim1'),
                        ('tlx3','tlx3'),
                        ('unknown', 'unknown'))
        else:
            raise Exception('variable not defined')

    elif catagory_type=='layer':
        col_name=('pre_layer', 'post_layer')
        if variable=='NRMSE':
            if fit_type=='excellent':
                values=(('2', '2'),
                       ('2/3', '2/3'),
                       ('3', '3'),
                       ('4', '4'),
                       ('5', '5'))
            elif fit_type=='good':
                values=(('2', '2'),
                       ('2/3', '2/3'),
                       ('3', '3'),
                       ('4', '4'),
                       ('5', '5'))
        elif variable=='amp':
            if fit_type=='excellent':
                values=(('2', '2'),
                       ('2/3', '2/3'),
                       ('3', '3'),
                       ('4', '4'),
                       ('5', '5'),
                       ('6', '6'))
            elif fit_type=='good':
                values=(('2', '2'),
                       ('2/3', '2/3'),
                       ('3', '3'),
                       ('4', '4'),
                       ('5', '5'),
                       ('6', '6'))
        elif variable=='rise_time':
            if fit_type=='excellent':
                values=(('2', '2'),
                       ('2/3', '2/3'),
                       ('5', '5'))
            elif fit_type=='good':
                values=(('2', '2'),
                       ('2/3', '2/3'),
                       ('4', '4'),
                       ('5', '5'),
                       ('6', '6'))
        elif variable=='latency':
            if fit_type=='excellent':
                values=(('2', '2'),
                       ('2/3', '2/3'),
                       ('3', '3'),
                       ('4', '4'),
                       ('5', '5'),
                       ('6', '6'))
            elif fit_type=='good':
                values=(('2', '2'),
                       ('2/3', '2/3'),
                       ('3', '3'),
                       ('4', '4'),
                       ('5', '5'),
                       ('6', '6'))
        else:
            raise Exception('variable not defined')
        
    else:
        raise Exception('catagory doesnt exist')
        
    #for each cre catagory get a subset make a new column for catagory
    new_df=pd.DataFrame()
    for value in values: 
        cat_df=df[(df[col_name[0]]==value[0]) & (df[col_name[1]]==value[1])]
        key=catagory_type+'_catagory'
        v=value[0]+'_to_'+value[1]
        cat_df[key]=v
        # concatenate to whole dataFrame
#         print(new_df)
#         print(cat_df)
        new_df=pd.concat([new_df, cat_df], axis=0, join='outer', join_axes=None, ignore_index=True,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=False)
    return new_df

# EXCITATION

## Significance of regression for excitation catagories
#### Results
All parameters except decay tau have statistically significant regression lines for inhibitory and excitatory catagories for both 'excellent fit' data (shown on left) and 'well fit' data (shown on right)

In [ ]:
regression_significance([excellent_df, good_df], 'excitation', data_type='log_', variables=(('NRMSE_v', 'NRMSE_i'),
               ('amp_v', 'amp_i'),
               ('rise_time_v', 'rise_time_i')))
regression_significance([excellent_df, good_df], 'excitation', data_type='', variables=(
               ('latency_v', 'latency_i'),
               ('decay_tau_v', 'decay_tau_i')))  

## Are excitatory and inhibitory linear regressions significantly different?

### Results 
NRMSE is not significantly different in 'excellent fit' data but intercept is in 'well fit' data. <br>
AMPLITUDE slope is significantly different in both 'excellent fit' and 'well fit' data. <br>
RISE_TAU is significantly different and noticibly different by eye in both 'excellent fit' and 'well fit' data. <br>
LATENCY intercept is significantly difference but not the slope in both 'excellent fit' and 'well fit' data.

'Excellent fit' data shown on the left and 'well fit' data on the right. <br>  

In [ ]:
cat_sig_diff([excellent_df, good_df], 'excitation', data_type='log_', variables=(('NRMSE_v', 'NRMSE_i'),
               ('amp_v', 'amp_i'),
               ('rise_time_v', 'rise_time_i')))
cat_sig_diff([excellent_df, good_df], 'excitation', data_type='', variables=(
               ('latency_v', 'latency_i'),))
# not doing decay tau as it doesnt is not related
#               ('decay_tau_v', 'decay_tau_i')))


# CRE LINES

## Cre line groups available in the data
### Results
pv to pv, sim1 to sim1, and unknown to unknown have enough points in both excellent and good fits.

rorb to rorb and tlx3 to tlx3 are on the cusp with 6 data points each in the excellent data set and only a couple are added with the good data set. 

In [ ]:
# look at what is in data sets
print('excellent fits')
print(excellent_df.groupby(['pre_cre', 'post_cre']).size())

print('\ngood fits')
print(good_df.groupby(['pre_cre', 'post_cre']).size())

## Significance of regression for cre-line catagories
#### Results
NRMSE: pvalb, sim1, and unknown have a statistically significant slope in both 'excellent' and 'well' fit data.  Rorb is not statistically significant in either case and tlx3 to tlx3 is significant in 'excellent' and not in 'well'.<br>
AMPLITUDE: All catagories are statistically significant in 'well fit' data but Rorb and tlx3 are not significant in the 'excellent' fit data probably due to number of data points. <br>
RISE_TAU: Surprisingly only sim1 and unknown are statistically significant in both 'excellent' and 'well' fit data. <br>
LATENCY: All catagories are significant in both 'excellent' and 'well' fit data.

'Excellent fit' data shown on the left and 'well fit' data on the right. <br>  

In [ ]:
#plot all the data
regression_significance([excellent_df, good_df], 'cre', data_type='log_', variables=(
               ('NRMSE_v', 'NRMSE_i'),
               ('amp_v', 'amp_i'),
               ('rise_time_v', 'rise_time_i')))
regression_significance([excellent_df, good_df], 'cre', data_type='', variables=(
               ('latency_v', 'latency_i'),
               ('decay_tau_v', 'decay_tau_i'))) 

## Are cre-lines with statistically significant regression lines significantly different from one another?

### Results
Inconsistent results between 'excellent' and 'well' fit data. Sometimes intercepts were found to be statistically significant but never slopes.

'Excellent fit' data shown on the left and 'well fit' data on the right. 

In [ ]:
#Merging catagory and stat sig from above
excellent=create_catagory_df(excellent_df, 'cre', 'NRMSE', 'excellent')
good=create_catagory_df(good_df, 'cre', 'NRMSE', 'good')
cat_sig_diff([excellent, good], 'cre', data_type='log_', variables=(('NRMSE_v', 'NRMSE_i'),))

excellent=create_catagory_df(excellent_df, 'cre', 'amp', 'excellent')
good=create_catagory_df(good_df, 'cre', 'amp', 'good')
cat_sig_diff([excellent, good], 'cre', data_type='log_', variables=(('amp_v', 'amp_i'),))

excellent=create_catagory_df(excellent_df, 'cre', 'rise_time', 'excellent')
good=create_catagory_df(good_df, 'cre', 'rise_time', 'good')
cat_sig_diff([excellent, good], 'cre', data_type='log_', variables=(('rise_time_v', 'rise_time_i'),))

excellent=create_catagory_df(excellent_df, 'cre', 'latency', 'excellent')
good=create_catagory_df(good_df, 'cre', 'latency', 'good')
cat_sig_diff([excellent, good], 'cre', data_type='', variables=(('latency_v', 'latency_i'),))

# LAYERS

## Layer groups available in the data

In [ ]:
# look at what is in data sets
print('excellent fits')
print(excellent_df.groupby(['pre_layer', 'post_layer']).size())

print('\ngood fits')
print(good_df.groupby(['pre_layer', 'post_layer']).size())

## Regression significance for different layer catagories
### Results
NRMSE: all except layer 6 are significant in both 'excellent' and 'well' fit. <br>
AMPLITUDE:  all are significant in both 'excellent' and 'well' fit. <br>
RISE_TAU: 2, 2/3, and 5 are significant in 'excellent' fit. In 'well' all except 3 are significant. <br>
LATENCY: all are significant in both 'excellent' and 'well' fit. <br>

'Excellent fit' data shown on the left and 'well fit' data on the right. <br>  

In [ ]:
#plot all the data
regression_significance([excellent_df, good_df], 'layer', data_type='log_', variables=(
               ('NRMSE_v', 'NRMSE_i'),
               ('amp_v', 'amp_i'),
               ('rise_time_v', 'rise_time_i')))
regression_significance([excellent_df, good_df], 'layer', data_type='', variables=(
               ('latency_v', 'latency_i'),
               ('decay_tau_v', 'decay_tau_i'))) 

## Are layers with statistically significant regression lines significantly different from one another?

In [ ]:
#Merging catagory and stat sig from above
excellent=create_catagory_df(excellent_df, 'layer', 'NRMSE', 'excellent')
good=create_catagory_df(good_df, 'layer', 'NRMSE', 'good')
cat_sig_diff([excellent, good], 'layer', data_type='log_', variables=(('NRMSE_v', 'NRMSE_i'),))

excellent=create_catagory_df(excellent_df, 'layer', 'amp', 'excellent')
good=create_catagory_df(good_df, 'layer', 'amp', 'good')
cat_sig_diff([excellent, good], 'layer', data_type='log_', variables=(('amp_v', 'amp_i'),))

excellent=create_catagory_df(excellent_df, 'layer', 'rise_time', 'excellent')
good=create_catagory_df(good_df, 'layer', 'rise_time', 'good')
cat_sig_diff([excellent, good], 'layer', data_type='log_', variables=(('rise_time_v', 'rise_time_i'),))

excellent=create_catagory_df(excellent_df, 'layer', 'latency', 'excellent')
good=create_catagory_df(good_df, 'layer', 'latency', 'good')
cat_sig_diff([excellent, good], 'layer', data_type='', variables=(('latency_v', 'latency_i'),))

# Images of the voltage (left) and current clamp (right) of 'excellent' fits

In [ ]:
from IPython.display import Image, display 
import matplotlib.image as mpimg
for p in excellent_df[['image_path_i', 'image_path_v']].iterrows():
    f=plt.figure(figsize=(20,10))
    f.add_subplot(121)
    plt.imshow(mpimg.imread(p[1].image_path_i))
    plt.axis('off')
    f.add_subplot(122)
    plt.imshow(mpimg.imread(p[1].image_path_v)) 
    plt.axis('off')
    plt.show()

# Images of the voltage (left) and current clamp (right) of 'well' fit data

In [ ]:
only_good_df=merged_df[(((merged_df['good_fit_i']=='excellent') | (merged_df['good_fit_i']=='good')) & 
                       ((merged_df['good_fit_v']=='excellent') | (merged_df['good_fit_v']=='good')) & 
                       ((merged_df['data_clarity_v']=='well') | (merged_df['data_clarity_v']=='ok')) & 
                       ((merged_df['data_clarity_i']=='well') | (merged_df['data_clarity_i']=='ok'))) &
                       ~(((merged_df['good_fit_i']=='excellent') & (merged_df['good_fit_v']=='excellent')) & 
                      ((merged_df['data_clarity_v']=='well') & (merged_df['data_clarity_i']=='well')))]
for p in only_good_df[['image_path_i', 'image_path_v']].iterrows():
    f=plt.figure(figsize=(20,10))
    f.add_subplot(121)
    plt.imshow(mpimg.imread(p[1].image_path_i))
    plt.axis('off')
    f.add_subplot(122)
    plt.imshow(mpimg.imread(p[1].image_path_v)) 
    plt.axis('off')
    plt.show()


# Images of the voltage (left) and current clamp (right) of other fits

In [ ]:
other_df=merged_df[~(((merged_df['good_fit_i']=='excellent') | (merged_df['good_fit_i']=='good')) & 
                       ((merged_df['good_fit_v']=='excellent') | (merged_df['good_fit_v']=='good')) & 
                       ((merged_df['data_clarity_v']=='well') | (merged_df['data_clarity_v']=='ok')) & 
                       ((merged_df['data_clarity_i']=='well') | (merged_df['data_clarity_i']=='ok'))) &
                       ~(((merged_df['good_fit_i']=='excellent') & (merged_df['good_fit_v']=='excellent')) & 
                      ((merged_df['data_clarity_v']=='well') & (merged_df['data_clarity_i']=='well')))]
for p in other_df[['image_path_i', 'image_path_v']].iterrows():
    f=plt.figure(figsize=(20,10))
    f.add_subplot(121)
    try:
        plt.imshow(mpimg.imread(p[1].image_path_i))
        plt.axis('off')
        f.add_subplot(122)
        plt.imshow(mpimg.imread(p[1].image_path_v)) 
        plt.axis('off')
        plt.show()
    except:
        pass